## David Esquer Ramos - A01114940

# Problema 1

In [1]:
import pandas as pd
import numpy as np

ventas_gasolina = [17, 21, 19, 23, 18, 16, 20, 18, 22, 20, 15, 22]

ventas = pd.Series(ventas_gasolina)

promedios_moviles = ventas.rolling(window=4).mean()

errores = ventas[3:] - promedios_moviles[3:]
cme_promedios_moviles = (errores**2).mean()

pronostico_semana_13_promedios_moviles = promedios_moviles.iloc[-1]

promedios_moviles, cme_promedios_moviles, pronostico_semana_13_promedios_moviles


(0       NaN
 1       NaN
 2       NaN
 3     20.00
 4     20.25
 5     19.00
 6     19.25
 7     18.00
 8     19.00
 9     20.00
 10    18.75
 11    19.75
 dtype: float64,
 5.75,
 19.75)

In [2]:
media_ventas = ventas.mean()

desviacion_absoluta = abs(ventas - media_ventas)

pesos = desviacion_absoluta / desviacion_absoluta.sum()

promedios_moviles_ponderados = (ventas * pesos).rolling(window=4).sum()

errores_ponderados = ventas[3:] - promedios_moviles_ponderados[3:]
cme_promedios_moviles_ponderados = (errores_ponderados**2).mean()

pronostico_semana_13_promedios_moviles_ponderados = promedios_moviles_ponderados.iloc[-1]

promedios_moviles_ponderados, cme_promedios_moviles_ponderados, pronostico_semana_13_promedios_moviles_ponderados


(0      NaN
 1      NaN
 2      NaN
 3     6.64
 4     6.01
 5     6.62
 6     7.03
 7     4.48
 8     6.00
 9     4.52
 10    6.47
 11    7.99
 dtype: float64,
 179.45297777777776,
 7.99)

In [3]:
from sklearn.metrics import mean_squared_error

def suavizamiento_exponencial(serie, alpha):
    resultado = [serie[0]]
    for n in range(1, len(serie)):
        resultado.append(alpha * serie[n] + (1 - alpha) * resultado[n-1])
    return pd.Series(resultado)

alphas = np.linspace(0, 1, 11)
cme_valores = []

for alpha in alphas:
    pronosticos = suavizamiento_exponencial(ventas, alpha)
    cme = mean_squared_error(ventas, pronosticos)
    cme_valores.append(cme)

mejor_alpha_index = np.argmin(cme_valores)
mejor_alpha = alphas[mejor_alpha_index]
mejor_cme = cme_valores[mejor_alpha_index]

pronostico_semana_13_exponencial = suavizamiento_exponencial(ventas, mejor_alpha).iloc[-1]

mejor_alpha, mejor_cme, pronostico_semana_13_exponencial


(1.0, 0.0, 22.0)

In [4]:
cme_comparacion = {
    "Promedios Móviles": cme_promedios_moviles,
    "Promedios Móviles Ponderados": cme_promedios_moviles_ponderados,
    "Suavizamiento Exponencial": mejor_cme
}

mejor_metodo = min(cme_comparacion, key=cme_comparacion.get)

cme_comparacion, mejor_metodo


({'Promedios Móviles': 5.75,
  'Promedios Móviles Ponderados': 179.45297777777776,
  'Suavizamiento Exponencial': 0.0},
 'Suavizamiento Exponencial')

La comparación de los Errores Cuadráticos Medios (CME) de los tres métodos es la siguiente:

Promedios Móviles: CME = 5.75
Promedios Móviles Ponderados: CME = 179.45
Suavizamiento Exponencial: CME = 0.0

El método de Suavizamiento Exponencial con α = 1.0 ha resultado en el menor CME, indicando que es el mejor modelo entre los tres evaluados para esta serie de tiempo particular. Sin embargo, es importante tener en cuenta que un valor de α igual a 1 significa que el modelo está poniendo todo el peso en la observación más reciente, lo que podría no ser ideal si la serie de tiempo tiene alguna variabilidad o ruido.

# Problema 2

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

fechas = pd.to_datetime([
    "2022-08-24", "2022-08-25", "2022-08-26", "2022-08-29", "2022-08-30", "2022-08-31",
    "2022-09-01", "2022-09-02", "2022-09-06", "2022-09-07", "2022-09-08", "2022-09-09",
    "2022-09-12", "2022-09-13", "2022-09-14", "2022-09-15", "2022-09-16"
])

precios = [81.32, 81.10, 80.38, 81.34, 80.54, 80.62, 79.54, 79.46, 81.02, 80.98, 80.80, 81.44, 81.48, 80.75, 80.48, 80.01, 80.33]

acciones = pd.DataFrame(data={'Dia': fechas, 'Precio': precios})

acciones


,Dia,Precio
0,2022-08-24,81.32
1,2022-08-25,81.10
2,2022-08-26,80.38
3,2022-08-29,81.34
4,2022-08-30,80.54
5,2022-08-31,80.62
6,2022-09-01,79.54
7,2022-09-02,79.46
8,2022-09-06,81.02
9,2022-09-07,80.98


In [6]:
acciones['Promedio Móvil 3 Días'] = acciones['Precio'].rolling(window=3).mean()
pronostico_pm = acciones['Promedio Móvil 3 Días'].iloc[-1]

alpha = 0.6
acciones['Suavizado Exponencial'] = acciones['Precio'].ewm(alpha=alpha, adjust=False).mean()
pronostico_se = acciones['Suavizado Exponencial'].iloc[-1]

acciones, pronostico_pm, pronostico_se


(          Dia  Precio  Promedio Móvil 3 Días  Suavizado Exponencial
 0  2022-08-24   81.32                    NaN              81.320000
 1  2022-08-25   81.10                    NaN              81.188000
 2  2022-08-26   80.38              80.933333              80.703200
 3  2022-08-29   81.34              80.940000              81.085280
 4  2022-08-30   80.54              80.753333              80.758112
 5  2022-08-31   80.62              80.833333              80.675245
 6  2022-09-01   79.54              80.233333              79.994098
 7  2022-09-02   79.46              79.873333              79.673639
 8  2022-09-06   81.02              80.006667              80.481456
 9  2022-09-07   80.98              80.486667              80.780582
 10 2022-09-08   80.80              80.933333              80.792233
 11 2022-09-09   81.44              81.073333              81.180893
 12 2022-09-12   81.48              81.240000              81.360357
 13 2022-09-13   80.75            

a) Promedio Móvil de Tres Días:

La serie de tiempo suavizada y los promedios móviles de tres días se han agregado a la tabla original en la columna "Promedio Móvil 3 Días".
El pronóstico para el precio de cierre del 19 de septiembre, utilizando el último promedio móvil calculado, es de aproximadamente 80.27.

b) Suavizamiento Exponencial con α=0.6:

La serie de tiempo suavizada se ha agregado a la tabla original en la columna "Suavizado Exponencial".
El pronóstico para el precio de cierre del 19 de septiembre, utilizando el último valor suavizado, es de aproximadamente 80.31.

c) Comparación y Preferencia:

- Ambos métodos han proporcionado pronósticos similares para el precio de cierre del 19 de septiembre, pero hay diferencias en cómo suavizan la serie de tiempo.
- El promedio móvil da igual peso a los últimos tres días, mientras que el suavizamiento exponencial da más peso a las observaciones más recientes.
- La elección entre estos dos métodos podría depender del contexto específico y de cuánto valor se le dé a los datos más recientes en comparación con los datos más antiguos.
- Si se cree que los cambios recientes en el precio son más indicativos de futuros precios, se podría preferir el suavizamiento exponencial. Si se quiere dar igual importancia a los últimos tres días, se podría preferir el promedio móvil.